In [1]:
from config import *
isaac_data_folder = parentdir + "/data/isaac/"
import pandas as pd
from main_package.utils import data_path_to_abs_path
from json import dump

chunk processing

In [7]:
chunksize = 10**4
df_chunks = pd.read_csv(data_path_to_abs_path("isaac/sample_question_attempts", file_type="txt"), chunksize=chunksize)

In [3]:
chunksize = 10**7 #the output csv will have around 0.25GB
df_chunks = pd.read_csv(data_path_to_abs_path("isaac/all_unprocessed_interaction_data/de-identified_question_attempts.csv"), chunksize=chunksize)

In [15]:
user_hash_to_user_id_dict = {}
count = 0
for df_chunk in df_chunks:
    print(f"processing chunk number {count}")
    df_chunk = df_chunk[df_chunk['attempt_no'] == 1]
    for user_hash in df_chunk['user_id'].unique():
        if user_hash not in user_hash_to_user_id_dict:
            user_hash_to_user_id_dict[user_hash] = str(len(user_hash_to_user_id_dict))
    df_chunk.loc[:, "user_id"] = df_chunk['user_id'].map(user_hash_to_user_id_dict)
    df_chunk = df_chunk.drop(columns='attempt_no')
    print("writing to csv file")
    # df_chunk.to_csv(data_path_to_abs_path(f"isaac/interaction_data/part_{count}.csv"), index=False)
    count += 1
    print("chunk successfuly processed")
with open(data_path_to_abs_path(f"isaac/interaction_data/dict.json"), "w") as file:
    dump(user_hash_to_user_id_dict, file, indent=2)

processing chunk number 0
processing chunk number 1
processing chunk number 2
processing chunk number 3


all at once

In [4]:
df = pd.read_csv(data_path_to_abs_path("isaac/all_unprocessed_interaction_data/de-identified_question_attempts.csv"))

In [5]:
user_hash_to_user_id_dict = {}
count = 0
df_chunk = df
df_chunk = df_chunk[df_chunk['attempt_no'] == 1]
for user_hash in df_chunk['user_id'].unique():
    if user_hash not in user_hash_to_user_id_dict:
        user_hash_to_user_id_dict[user_hash] = str(len(user_hash_to_user_id_dict))
df_chunk.loc[:, "user_id"] = df_chunk['user_id'].map(user_hash_to_user_id_dict)
df_chunk = df_chunk.drop(columns='attempt_no')
print("writing to csv file")
df_chunk.to_csv(data_path_to_abs_path(f"isaac/interaction_data/all_data.csv"), index=False)
count += 1
print("chunk successfuly processed")
#with open(data_path_to_abs_path(f"isaac/interaction_data/dict.json"), "w") as file:
    #dump(user_hash_to_user_id_dict, file, indent=2)

/tmp/ipykernel_1538/1758268286.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk.loc[:, "user_id"] = df_chunk['user_id'].map(user_hash_to_user_id_dict)


writing to csv file
chunk successfuly processed


In [6]:
df.head()

,user_id,question_id,correct,attempt_no
0,9b5738638d4eb16007ed611e683ed6632aa60f93f69b63...,vector_scalar_train|80fbaf79-a3f4-4a4b-a318-00...,f,1
1,9b5738638d4eb16007ed611e683ed6632aa60f93f69b63...,vector_scalar_train|80fbaf79-a3f4-4a4b-a318-00...,f,2
2,6cb8d32dfc6ace770d521d2930995910662e2aaca7782b...,ch_a_p3|dc864852-f371-4f6d-a87d-d87f982bb0f7,t,1
3,9b5738638d4eb16007ed611e683ed6632aa60f93f69b63...,vector_scalar_train|80fbaf79-a3f4-4a4b-a318-00...,f,3
4,9b5738638d4eb16007ed611e683ed6632aa60f93f69b63...,vector_scalar_train|80fbaf79-a3f4-4a4b-a318-00...,f,4


In [ ]:
df_chunk['correct'] = df_chunk['correct'] == 't'
df_chunk['correct'] = df_chunk['correct'].astype(int)
df_chunk = df_chunk.rename(columns={'question_id': "sequence_id"})
df_chunk = df_chunk.reset_index().rename(columns={df.index.name: 'log_id'})
df_chunk.head()

In [8]:
df_chunk[:1500].to_csv(data_path_to_abs_path(f"isaac/interaction_data/all_data_sample.csv"), index=False)